In [1]:
"""
Data Mining Matmatical Problem Solver
- Decision Tree
    -ID3
Submission Date: 28 Feb 21
"""
__author__='Black D Chase'
__version__='0.3.1'

In [2]:
# Imports Libraries
import math
import pandas as pd
import numpy as np
import sys, os
import graphviz

In [3]:
class Node():
    def __init__(self, index = None, children = [], entropy = 0, depth = 0):
        self.index = index
        self.entropy = entropy
        self.depth = depth
        self.attribute = None
        self.children = children
        self.order = None
        self.label = None

    def getName(self):
        #"Attribute: "+str(self.attribute)
        self.name=str(self.order)+" "+str(self.attribute)+"\n "+str(self.entropy)
        return self.name
    
    def __hash__(self):
        return hash(self.getName())

    def __eq__(self,other):
        if type(self)==type(other):
            return self.getName()==other.getName()
        return self.name==other
    
    def setProperties(self, attribute, order):
        self.attribute = attribute
        self.order = order

    def set_label(self, label):
        self.label = label

In [4]:
class DecisionTreeID3():
    def __init__(self):
        self.root = None
        self.minGain = 1e-2
        self.graph = graphviz.Digraph()

    def fit(self, data, target):
        self.Ntrain = data.count()[0]
        self.data = data
        self.attributes = list(data)
        self.max_depth = len(self.attributes)*2
        self.target = target
        self.labels = target.unique()

        index = range(self.Ntrain)
        self.root = Node(index = index, entropy = self.calEntropy(index), depth = 0)
        queue = [self.root]
        while queue:
            node = queue.pop()
            if node.depth < self.max_depth or node.entropy < self.minGain:
                node.children = self.makePartition(node)
                if not node.children: #leaf node
                    self._set_label(node)
                queue += node.children
            else:
                self._set_label(node)

    def calEntropy(self, index):
        if len(index) == 0:
            return 0
        index = [i for i in index]
        freq = np.array(self.target.iloc[index].value_counts())
        return self.entropy(freq)

    def entropy(self,freq):
        freq_0 = freq[np.array(freq).nonzero()[0]]
        prob_0 = freq_0/float(freq_0.sum())
        return -np.sum(prob_0*np.log(prob_0))

    def _set_label(self, node):
        target_index = [i + 1 for i in node.index]
        node.set_label(self.target[target_index].mode()[0])

    def makePartition(self, node):
        index = node.index
        maxGain = 0
        bestmakePartitions = []
        decisionAttribute = None
        order = None
        sub_data = self.data.iloc[index, :]
        for i, att in enumerate(self.attributes):
            values = self.data.iloc[index, i].unique().tolist()
            if len(values) == 1:
                continue
            splits = []
            for val in values:
                sub_index = sub_data.index[sub_data[att] == val].tolist()
                splits.append([sub_id-1 for sub_id in sub_index])
            if min(map(len, splits)) < max(self.Ntrain//10000,2):
                continue
            HxS= 0
            for split in splits:
                HxS += len(split)*self.calEntropy(split)/len(index)
            gain = node.entropy - HxS
            if gain < self.minGain:
                continue
            if gain > maxGain:
                maxGain = gain
                bestmakePartitions = splits
                decisionAttribute = att
                order = values
            print(f"Attribut: {att}, Gain: {gain}, HxS: {HxS}")
        if decisionAttribute is None:
            print(f"No further Partitioning for {node.attribute}")
        else:
            print(f"Descision Attribute: {decisionAttribute}")
        node.setProperties(decisionAttribute, order)
        
        child_nodes = [Node(
            index = split,
            entropy = self.calEntropy(split),
            depth = node.depth + 1,
        ) for split in bestmakePartitions]
        
        return child_nodes

    def predict(self, new_data):
        npoints = new_data.count()[0]
        labels = [None]*npoints
        for n in range(npoints):
            x = new_data.iloc[n, :] # one point 
            # start from root and recursively travel if not meet a leaf 
            node = self.root
            while node.children:
                node = node.children[node.order.index(x[node.attribute])]
            labels[n] = node.label
        return labels
    
    
    def getTree(self):
        queue = [self.root]
        visited = {}
        while len(queue)>0:
            node = queue.pop()
            visited[node]= node
            for i in node.children:
                parent=node.getName()
                child=i.getName()
                self.graph.edge(parent,child)
                if i not in visited.keys():
                    queue.append(i)

    def showTree(self):
        self.graph.render("OutputID3")
        return self.graph.view()
    


In [5]:
''' If has no column name,add like this
columns = ["buying", "maint", "doors", "persons", "lugboot", "safety"]
df = pd.read_csv('car_data.csv', names=columns)
df = df.reset_index()
del df["index"]
#'''
df = pd.read_csv('testID3.csv')
print(df.head(10))

   Index Column1 Column2 Column3 Column4 Target
0      0     40>    Fast    High    Good    Yes
1      1     40>    Slow     Low    Good    Yes
2      2     40>    Slow     Low     Bad     No
3      3     >40    Fast    High     Bad     No
4      4     >40    Fast     Low     Bad    Yes
5      5     >40    SLow    High    Good     No
6      6     >40    Fast    High    Good     No
7      7     40>    Fast     Low     Bad     No


In [6]:
X = df.iloc[:, :-1]
X.head(10)

,Index,Column1,Column2,Column3,Column4
0,0,40>,Fast,High,Good
1,1,40>,Slow,Low,Good
2,2,40>,Slow,Low,Bad
3,3,>40,Fast,High,Bad
4,4,>40,Fast,Low,Bad
5,5,>40,SLow,High,Good
6,6,>40,Fast,High,Good
7,7,40>,Fast,Low,Bad


In [7]:
y = df.iloc[:, -1]
y.head(10)

0    Yes
1    Yes
2     No
3     No
4    Yes
5     No
6     No
7     No
Name: Target, dtype: object

In [8]:
tree = DecisionTreeID3()
tree.fit(X, y)
tree.getTree()
yPredict = tree.predict(X)
yPredict

Attribut: Column1, Gain: 0.03382207556860539, HxS: 0.6277411625893767
Attribut: Column3, Gain: 0.03382207556860539, HxS: 0.6277411625893767
Attribut: Column4, Gain: 0.03382207556860539, HxS: 0.6277411625893767
Descision Attribute: Column1
No further Partitioning for None
Attribut: Column3, Gain: 0.34657359027997264, HxS: 0.34657359027997264
Descision Attribute: Column3
No further Partitioning for None
No further Partitioning for None


['No', 'No', 'No', 'No', 'No', 'No', 'No', 'No']

In [9]:
acc = (yPredict==y).values.sum()/len(y)
print("Accuracy is of",acc*100)

Accuracy is of 62.5


In [10]:
tree.showTree()

'OutputID3.pdf'